<a href="https://colab.research.google.com/github/afirth03/IMLOAssessment/blob/main/IMLO_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import optim as optim
import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation = 0.2, hue = 0.1),
    transforms.RandomAffine(degrees = 0, translate = (0.1,0.1), scale=(0.9, 1.1), shear = 10),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.1, scale = (0.01, 0.1), ratio=(0.9, 1.1)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

batch_size = 64

train_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform=transform_train
)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform=transform

)

testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                          shuffle=False, num_workers=0)

val_data = datasets.Flowers102(
    root="data",
    split="val",
    download=True,
    transform=transform
)

valloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size,
                                          shuffle=False, num_workers=0)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.features = nn.Sequential( #Sequential for modularity

            nn.Conv2d(3, 64, 5, stride = 2, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, 5, stride = 2, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(5, 2),

            nn.Conv2d(128, 256, 3, stride = 1, padding = 1), #Reduced stride from 2 to 1 to capture detailed info from images
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(5, 2),

            nn.Conv2d(256, 512, 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(5, 2),

            nn.AdaptiveAvgPool2d((14, 14)) #Makes the input fit a 512 * 14 * 14 input

        )

        self.classifier = nn.Sequential(
            nn.Linear(512 * 14 * 14, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5), #Dropout data to reduce overfitting
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.25),
            nn.Linear(512, 102)  #Output classes in Flowers 102
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  #Flatten the output for the fully connected layer
        x = self.classifier(x)
        return x

cnn = CNN()
print(cnn)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn.to(device)

In [ ]:
cnn = CNN()
cnn.load_state_dict(torch.load('cnn_flowers102.pth', map_location=device))

In [ ]:
cnn.eval()
running_loss = 0.0
running_accuracy = 0.0
criterion = nn.CrossEntropyLoss()

for i, data in enumerate(valloader):
    inputs, labels = data[0].to(device), data[1].to(device)

    with torch.no_grad():
        outputs = cnn(inputs)
        correct = torch.sum(labels == torch.argmax(outputs, dim=1)).item()
        running_accuracy += correct / batch_size
        loss = criterion(outputs, labels)
        running_loss += loss.item()

avg_loss_across_batches = running_loss / len(valloader)
avg_acc_across_batches = (running_accuracy / len(valloader)) * 100

print(f'Val Loss: {avg_loss_across_batches:.3f}, Val Accuracy: {avg_acc_across_batches:.1f}%')

In [ ]:
cnn.eval()

with torch.no_grad():
    correct = 0
    total = 0

    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)

        outputs = cnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test images: {accuracy:.2f}%')